# Run analysis on the data

In [1]:
%matplotlib qt

In [2]:
%run -i ~/Documents/tomology/tomography/utils.py

/home/xf28id2/conda_envs/khalifah_20210725/lib/python3.9/site-packages/pims/image_reader.py:26: RuntimeWarning: PIMS image_reader.py could not find scikit-image. Falling back to matplotlib's imread(), which uses floats instead of integers. This may break your scripts. 
(To ignore this warning, include the line "warnings.simplefilter("ignore", RuntimeWarning)" in your script.)
  warnings.warn(RuntimeWarning(ski_preferred))


In [24]:
import pyFAI
import pathlib
import matplotlib as mpl

## Create the calculator

In [4]:
C = Calculator()

## Load the Run Data

In [10]:
from databroker import catalog

db = catalog["xpdd"]

In [11]:
UID = 'fef1f2f7-301c-41c6-a775-d89e1fe0af23'
run = db[UID]
C.metadata = dict(run.metadata["start"])

In [12]:
C.metadata["shape"] = C.metadata["shape"][1:]
C.metadata["extents"] = C.metadata["extents"][1:]

## Load the processed data

In [14]:
dataset = xr.load_dataset("/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/CG-0046_fef1f2f7-301c-41c6-a775-d89e1fe0af23.nc")

In [15]:
dataset

<xarray.Dataset>
Dimensions:    (pixel_y: 3888, pixel_x: 3072, grain: 80, dim_0: 61, dim_1: 29)
Coordinates:
  * grain      (grain) int32 45 10 18 50 37 42 63 81 ... 53 90 71 0 47 80 38 84
  * dim_0      (dim_0) float64 28.0 28.25 28.5 28.75 ... 42.25 42.5 42.75 43.0
  * dim_1      (dim_1) float64 61.0 61.25 61.5 61.75 ... 67.25 67.5 67.75 68.0
Dimensions without coordinates: pixel_y, pixel_x
Data variables:
    dark       (pixel_y, pixel_x) int16 317 316 313 319 313 ... 327 314 311 320
    light      (pixel_y, pixel_x) int16 14 15 15 14 31 13 ... 16 15 13 15 14 15
    intensity  (grain, dim_0, dim_1) float64 533.0 542.9 374.1 ... 327.5 327.4
    y          (grain) int32 2265 485 803 2488 1524 ... 44 2346 3323 1663 3362
    dy         (grain) int32 40 40 40 40 40 40 40 40 ... 40 40 40 40 40 40 40 40
    x          (grain) int32 1352 1715 824 143 2614 ... 1870 699 1164 1267 42
    dx         (grain) int32 40 40 40 40 40 40 40 40 ... 40 40 40 40 40 40 40 40
    Q          (grain) float64 26.43 78.77 81.43 81.28 ... 59.62 39.89 97.82

In [35]:
C.intensity = dataset["intensity"].values
C.light = dataset["light"].values
C.dark = None
C.windows = dataset[["y", "dy", "x", "dx", "Q"]].to_dataframe()
C.calc_coords()

## Load the calibration data

In [8]:
PONI_FILE = "/nsls2/data/xpd/legacy/processed/from_mehmet/CeO2_0.25x0.25_beam.poni"
C.ai = pyFAI.load(PONI_FILE) if PONI_FILE else pyFAI.azimuthalIntegrator.AzimuthalIntegrator(dist=200, wavelength=0.186, detector="perkin")

## Visualize the windows

In [29]:
C.show_windows(norm=mpl.colors.TwoSlopeNorm(400))
plt.show()

## Visualize the intensity

In [49]:
facet = dataset["intensity"].plot(col="grain", col_wrap=20, norm=mpl.colors.TwoSlopeNorm(vcenter=500, vmin=300, vmax=800))
axes = facet.axes.flatten()
for i in range(C.intensity.shape[0]):
    d = 2 * np.pi / C.windows["Q"]
    axes[i].set_title("d = {:.3f} nm".format(d.iloc[i]))
    axes[i].invert_yaxis()
    axes[i].set_aspect("equal")
plt.show()